In [1]:
import pandas as pd
import numpy as np
import re
import tensorflow_datasets as tfds
import tensorflow as tf

## 데이터 선정 이유

- 현대 사회의 1인가구 비율은 높아지고 있음.
- 혼자 있는 시간에 필요한 대화용 챗봇을 만들어 보기로 함.

In [2]:
train = pd.read_csv('ChatBotData.csv')

In [3]:
train[300:310]

,Q,A,label
300,귀가 간지러,누가 욕하고 있나봐요.,0
301,귀가 윙윙거려,병원에 가세요.,0
302,귀농 어때?,생각하기는 쉬운데 실천하기는 어려운 것 같아요.,0
303,그 사람이 나 안 좋아하는 거 같아,슬픈 이야기네요.,0
304,그 사람이 나 좋아해줬으면 좋겠다,저도 간절히 기도 할게요.,0
305,그 사람이 행복했으면 좋겠다,그렇게 될 수 있을 거예요.,0
306,그 시절엔 다 그랬지,추억에 잠길 때도 필요해요.,0
307,그냥 고백할걸,후회는 후회를 낳을뿐이에요. 용기 내세요.,0
308,그냥 공무원이 좋을 듯,시작이 반이에요. 어서 준비하세요.,0
309,그냥 내버려 둬 주었으면,아무도 없는 곳으로 여행을 떠나보세요.,0


In [4]:
questions = []
for sentence in train['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [5]:
answers = []
for sentence in train['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [6]:
# subword text encoder 사용하여 질문, 답변 모두 포함한 단어집합 생성
# subword text encoder : 하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업.

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

#시작토큰, 종료토큰 변수 설정
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size + 2

In [7]:
MAX_LENGTH = 40

def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    
    #토큰화 후 인코딩, 시작 종료 토큰 추가
    for (sentence1, sentence2) in zip(inputs, outputs):
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
        
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    
    return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

## transformer model 구축

1. Positional Encoding
2. Scaled Dot Product
3. MultiHeadAttention
4. Padding Mask
5. Encoder
6. Decoder

## Positional Encoding

In [8]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
        
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000,(2*(i//2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                                    i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                                    d_model=d_model)
        
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        
        angle_rads = np.zeros(angle_rads.shape)
        angle_rads[:, 0::2] = sines
        angle_rads[:, 1::2] = cosines
        pos_encoding = tf.constant(angle_rads)
        pos_encoding = pos_encoding[tf.newaxis, ...]

        print(pos_encoding.shape)
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]    

## Scaled Dot Product

In [9]:
def scaled_dot_product_attention(query, key, value, mask):    
    
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    
    if mask is not None:
        logits += (mask * -1e9)
        
    attention_weights = tf.nn.softmax(logits, axis=-1)
    
    output = tf.matmul(attention_weights, value)
    
    return output, attention_weights    

## MultiHeadAttention

In [10]:
class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        
        self.dense = tf.keras.layers.Dense(units=d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs= tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0,2,1,3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        
        outputs = self.dense(concat_attention)
        
        return outputs

In [11]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x,0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

## Encoder Part

In [12]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    
    padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")
    
    attention = MultiHeadAttention(d_model, num_heads, name="attention")({'query': inputs, 'key': inputs, 'value' : inputs,
                                                                         'mask' : padding_mask})
    
    #dropout + 잔차 연결 층 정규화    
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    # position wise feed forward
    outputs = tf.keras.layers.Dense(units = dff, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units = d_model)(outputs)
    
    #dropout + 잔차 연결
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)
    
    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)   

In [13]:
def encoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    
    padding_mask = tf.keras.Input(shape=(1, 1, None), name = 'padding_mask')
    
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
                                dropout=dropout, name = "encoder_layer_{}".format(i),
                               )([outputs, padding_mask])
        
    return tf.keras.Model(inputs=[inputs, padding_mask], outputs = outputs, name=name)

## Decoder Part

In [14]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [15]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name = "inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    
    look_ahead_mask = tf.keras.Input(shape=(1,None, None), name = "look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')
    
    # 1.masked self attention(첫번째 서브층)
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(inputs={'query': inputs,
                                                                                   'key' : inputs,
                                                                                   'value': inputs,
                                                                                   'mask' : look_ahead_mask})
    #normalize
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)
    
    #2. decoder-encoder attention(두번째 서브층)
    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(inputs={'query': attention1,
                                                                                   'key' : enc_outputs,
                                                                                   'value': enc_outputs,
                                                                                   'mask' : padding_mask})
    
    #normalize
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)
    
    # 3. position wise feed forward(세번째 서브층)
    outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    
    #nomarlize
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)
    
    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

In [16]:
def decoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name='decoder'):
    inputs = tf.keras.Input(shape=(None, ), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name = 'look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1,1,None), name = 'padding_mask')
    
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads, dropout=dropout, 
                                name='decoder_layer_{}'.format(i),
                               )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])
        
    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

In [17]:
def transformer(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="transformer"):
    
    inputs = tf.keras.Input(shape=(None, ), name="inputs")
    
    dec_inputs = tf.keras.Input(shape=(None, ), name="dec_inputs")
    
    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask,
                                              output_shape= (1,1,None),name='enc_padding_mask')(inputs)
    
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask,
                                             output_shape=(1,None,None),name='look_ahead_mask')(dec_inputs)
    
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask,
                                              output_shape=(1,1,None), name = 'dec_padding_maks')(inputs)
    
    enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff, d_model=d_model, num_heads=num_heads,
                         dropout=dropout,)(inputs=[inputs, enc_padding_mask])
    
    dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff, d_model=d_model, num_heads=num_heads,
                         dropout=dropout,)(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])  
        
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
    
    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## loss_funtion, learning_rate_scheduler

In [18]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    return tf.reduce_mean(loss)

In [19]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps
        
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [20]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [21]:
tf.keras.backend.clear_session()

# hyper parameter 설정
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8180, 256)
(1, 8180, 256)


In [22]:
MAX_LENGTH = 40

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [23]:
EPOCHS = 100

model.fit(dataset, epochs=EPOCHS)

Epoch 1/100
185/185 [==============================] - 17s 66ms/step - loss: 1.4549 - accuracy: 0.0221
Epoch 2/100
185/185 [==============================] - 12s 66ms/step - loss: 1.1816 - accuracy: 0.0495
Epoch 3/100
185/185 [==============================] - 12s 66ms/step - loss: 1.0054 - accuracy: 0.0505
Epoch 4/100
185/185 [==============================] - 12s 66ms/step - loss: 0.9273 - accuracy: 0.0541
Epoch 5/100
185/185 [==============================] - 12s 65ms/step - loss: 0.8696 - accuracy: 0.0576
Epoch 6/100
185/185 [==============================] - 12s 65ms/step - loss: 0.8099 - accuracy: 0.0618
Epoch 7/100
185/185 [==============================] - 12s 64ms/step - loss: 0.7435 - accuracy: 0.0679
Epoch 8/100
185/185 [==============================] - 12s 64ms/step - loss: 0.6707 - accuracy: 0.0753
Epoch 9/100
185/185 [==============================] - 12s 65ms/step - loss: 0.5923 - accuracy: 0.0844
Epoch 10/100
185/185 [==============================] - 12s 64ms/step - l

185/185 [==============================] - 12s 64ms/step - loss: 0.0027 - accuracy: 0.1744
Epoch 80/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0026 - accuracy: 0.1744
Epoch 81/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0026 - accuracy: 0.1744
Epoch 82/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0027 - accuracy: 0.1743
Epoch 83/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0025 - accuracy: 0.1744
Epoch 84/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0025 - accuracy: 0.1744
Epoch 85/100
185/185 [==============================] - 12s 64ms/step - loss: 0.0025 - accuracy: 0.1744
Epoch 86/100
185/185 [==============================] - 12s 65ms/step - loss: 0.0024 - accuracy: 0.1744
Epoch 87/100
185/185 [==============================] - 12s 65ms/step - loss: 0.0024 - accuracy: 0.1744
Epoch 88/100
185/185 [==============================] - 12s 64ms/step - loss:

In [24]:
# 예측 시 전처리 수행
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [25]:
def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
    output = tf.expand_dims(START_TOKEN, 0)
    
    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)
        
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
            
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)

In [26]:
def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])
    
    print("\n")

    print('답변: {}'.format(predicted_sentence))

In [28]:
sen = input("대화를 입력하세요~")
predict(sen)

대화를 입력하세요~프로젝트 어려워..


답변: 성공적으로 끝날 거예요 .


In [29]:
sen = input("대화를 입력하세요~")
predict(sen)

대화를 입력하세요~배고프다


답변: 저도 밥 먹고 싶어요


In [30]:
sen = input("대화를 입력하세요~")
predict(sen)

대화를 입력하세요~어쩔티비


답변: 못 당해요 .  무시하세요 .


In [31]:
sen = input("대화를 입력하세요~")
predict(sen)

대화를 입력하세요~노래 추천해줘


답변: 좀 더 밝고 신나는 노래 들어보세요 .


In [32]:
sen = input("대화를 입력하세요~")
predict(sen)

대화를 입력하세요~날씨가 너무 더워


답변: 시원한 물이라도 한 잔 드세요~
